In [20]:
import requests
import json

In [21]:
'''
All roasters : https://classes.cornell.edu/api/2.0/config/rosters.json
All subs in Fall 14 : https://classes.cornell.edu/api/2.0/config/subjects.json?roster=FA14
roster=FA14

All Math classes in Fall 2014, use method search/classes
Link : https://classes.cornell.edu/api/2.0/search/classes.json?roster=FA14&subject=MATH
roster=FA14
subject=MATH
'''

response_roasters = requests.get("https://classes.cornell.edu/api/2.0/config/rosters.json")

In [22]:
print(response_roasters.status_code)
data = response_roasters.json()

pretty_json = json.dumps(data, indent=4)


200


In [23]:
print(pretty_json)

{
    "status": "success",
    "data": {
        "rosters": [
            {
                "slug": "FA14",
                "isDefaultRoster": "N",
                "strm": "2573",
                "descr": "Fall 2014",
                "descrshort": "2014FA",
                "attributeSrc": "CATALOG",
                "defaultSessionCode": "1",
                "defaultCampus": "MAIN",
                "defaultLocation": "ITH",
                "defaultInstructionMode": "P",
                "sharing": "N",
                "archiveMode": "Y",
                "version": {
                    "status": "COMPLETE",
                    "referenceDttm": "2015-01-14T18:16:01-0500",
                    "catalogDttm": "2015-01-14T18:21:04-0500",
                    "descriptionSource": "CATALOG",
                    "showCatalogNote": "Y",
                    "catalogCourseNote": null,
                    "catalog": {
                        "descrshort": "CoS 14-15",
                        "descr":

In [24]:
import time

In [32]:
BASE_URL = "https://classes.cornell.edu/api/2.0"


def fetch_rosters():
    """
    This function accesses Cornell API and returns the JSON response.

    Args:

    Returns: Returns the JSON of all the rosters.
        type: JSON object.
    """
    endpoint = "/config/rosters.json"
    response = requests.get(BASE_URL + endpoint)
    print(response.status_code)
    time.sleep(1)
    return response.json()  # Assuming API returns JSON. Use .text if XML.

def fetch_subjects(roster):
    """
    This function accesses particular roster and returns the JSON response.

    Args: Roster

    Returns: Returns the JSON of all subjects in particular roster.
        type: JSON object.
    """
    endpoint = f"/config/subjects.json?roster={roster}"
    response = requests.get(BASE_URL + endpoint)
    return response.json()

def fetch_classes_with_description(roster, subject):
    """
    This function accesses the particular roster and subject and return the classes associated with it alomg with description.

    Args: Roster, Subject

    Returns: Returns the JSON of all classes in particular roster and subject.
        type: JSON object.
    """
    endpoint = f"/search/classes.json?roster={roster}&subject={subject}"
    response = requests.get(BASE_URL + endpoint)
    return response.json()

def add_subjects_to_roster_in_json(data,subjects,term):
    """
    This function updates the final JSON object to add subjects to Roster.

    Args: JSON Object, subjects, term

    Returns: 
    """
    if isinstance(data, dict):
        for item in data["data"]:
            if item.get("Term")==term:
                subject_array = subjects["data"]["subjects"]
                new_json = {
                    "subjects": subject_array
                }
                item.update(new_json)
                
                
                
def get_class_info(description):
    """
    This function builds the classes array.

    Args: Description

    Returns: Returns the classes array.
        type: JSON object.
    """
    targetJSON ={
        "classes":[]
    }
    for class_info in description["data"]["classes"]:
        course_data={
            "titleShort": class_info["titleShort"],
            "titleLong": class_info["titleLong"],
            "description": class_info["description"]
        }
        targetJSON["classes"].append(course_data)
    return targetJSON
                
    
def add_classes_to_subjects_in_corrosponding_roster3(data,roster,subject_passed,description):
    """
    This function updates the final JSON object

    Args: Initial JSON object, roster, subject_passed, description

    Returns: 
    """
    for item in data['data']:
        if item['Term']==roster:
            for subject in item["subjects"]:
                if subject["value"]==subject_passed:
                    subject.update(description)
        

    
    
if __name__ == "__main__":
    rosters = fetch_rosters()
    #print(rosters)
    data={
        "data" :[]
    }
    counter=0;
    for roster in rosters['data']['rosters']:
        data["data"].append({
            "Year" : roster['descr'][-4:],        #basically accessing the last values to extract year
            "Term" :roster['slug'],
            "Term Description" : roster['descr']
        })
    
        subjects = fetch_subjects(roster['slug'])    #bascially the name of term
        add_subjects_to_roster_in_json(data,subjects,roster['slug'])
        
    #Building the JSON File    
    for roster in data["data"]:
        for subject in roster["subjects"]:
                json_description = fetch_classes_with_description(roster["Term"], subject["value"])
                class_data = get_class_info(json_description)
                add_classes_to_subjects_in_corrosponding_roster3(data,roster["Term"],subject["value"],class_data)
    with open('output.json', 'w') as f:
        json.dump(data, f, indent=4)
            

200


In [ ]:
import json

In [ ]:
with open('output.json', 'r') as f:
    json_data = json.load(f)

In [10]:
import pandas as pd
from pandas import json_normalize



In [11]:
# Normalize the top level
df_data = json_normalize(json_data, 'data')
df_data.head()

,Year,Term,Term Description,subjects
0,2014,FA14,Fall 2014,"[{'value': 'AAS', 'descr': 'Asian American Stu..."
1,2015,WI15,Winter 2015,"[{'value': 'AEM', 'descr': 'Applied Economics ..."
2,2015,SP15,Spring 2015,"[{'value': 'AAS', 'descr': 'Asian American Stu..."
3,2015,SU15,Summer 2015,"[{'value': 'AEM', 'descr': 'Applied Economics ..."
4,2015,FA15,Fall 2015,"[{'value': 'AAS', 'descr': 'Asian American Stu..."


In [12]:
# Normalize subjects (including nested classes)
df_subjects = json_normalize(json_data, record_path=['data','subjects'], 
                             meta=[['data', 'Year'], ['data', 'Term'], ['data', 'Term Description']], errors='ignore')

df_subjects.head()

,value,descr,descrformal,classes,data.Year,data.Term,data.Term Description
0,AAS,Asian American Studies,Asian American Studies,"[{'titleSort': 'Intro To Asian American Hist',...",2014,FA14,Fall 2014
1,AEM,Applied Economics & Management,Applied Economics & Management,[{'titleShort': 'FWS:Fd Systems in Devlpng Wrl...,2014,FA14,Fall 2014
2,AEP,Applied & Engineering Physics,Applied & Engineering Physics,"[{'titleShort': 'Laser & Photonics', 'titleLon...",2014,FA14,Fall 2014
3,AGSCI,Agriculture Sciences,Agricultural Sciences,"[{'titleShort': 'Exploring AGSCI Careers', 'ti...",2014,FA14,Fall 2014
4,AIRS,Air Force Science,Aerospace Studies,[{'titleShort': 'Foundations of US Air Force I...,2014,FA14,Fall 2014


In [14]:
# Normalize classes
df_classes = json_normalize(json_data, record_path=['data', 'subjects', 'classes'], 
                            meta=[['data', 'Year'], ['data', 'Term'], ['data', 'Term Description'], ['data', 'subjects', 'value']], errors='ignore')
df_classes.head(100)

,titleSort,titleLonitleShg,description,titleShort,titleLong,data.Year,data.Term,data.Term Description,data.subjects.value
0,Intro To Asian American Hist,Introduction to Asian American History,"An introductory history of Chinese, Japanese, ...",NaN,NaN,2014,FA14,Fall 2014,AAS
1,NaN,NaN,The common perception of ethnicity is that it ...,Asians in the Americas,Asians in the Americas: A Comparative Perspective,2014,FA14,Fall 2014,AAS
2,NaN,NaN,This course examines the experiences and repre...,Asian American Women's Hist,Asian American Women's History,2014,FA14,Fall 2014,AAS
3,NaN,NaN,,Independent Study,Independent Study,2014,FA14,Fall 2014,AAS
4,NaN,NaN,What is it like to be a subsistence farmer in ...,FWS:Fd Systems in Devlpng Wrld,FWS:Food Systems In The Developing World: Heal...,2014,FA14,Fall 2014,AEM
...,...,...,...,...,...,...,...,...,...
95,NaN,NaN,Laboratory or theoretical work in any branch o...,Independent Study for Honors,Independent Study for Honors,2014,FA14,Fall 2014,AEP
96,NaN,NaN,Topics include wave propagation in dielectrics...,Applied Electrodynamics,Applied Electrodynamics,2014,FA14,Fall 2014,AEP
97,NaN,NaN,Topics include plasma state; motion of charged...,Introduction to Plasma Physics,Introduction to Plasma Physics,2014,FA14,Fall 2014,AEP
98,NaN,NaN,Graduate-level introduction to the tools used ...,Nanocharacterization,Nanocharacterization,2014,FA14,Fall 2014,AEP


In [12]:
# Merging df_data and df_subjects
df_merged = pd.merge(df_data, df_subjects, left_on=['Year', 'Term', 'Term Description'], right_on=['data.Year', 'data.Term', 'data.Term Description'])

# Dropping redundant columns resulted from merging
df_merged = df_merged.drop(columns=['data.Year', 'data.Term', 'data.Term Description'])

# Merging df_merged and df_classes
df_final = pd.merge(df_merged, df_classes, left_on=['Year', 'Term', 'Term Description', 'value'], right_on=['data.Year', 'data.Term', 'data.Term Description', 'data.subjects.value'])

# Dropping redundant columns resulted from merging
df_final = df_final.drop(columns=['data.Year', 'data.Term', 'data.Term Description', 'data.subjects.value'])

df_final.head(15)

,Year,Term,Term Description,subjects,value,descr,descrformal,classes,titleShort,titleLong,description
0,2014,FA14,Fall 2014,"[{'value': 'AAS', 'descr': 'Asian American Stu...",AAS,Asian American Studies,Asian American Studies,[{'titleShort': 'Intro To Asian American Hist'...,Intro To Asian American Hist,Introduction to Asian American History,"An introductory history of Chinese, Japanese, ..."
1,2014,FA14,Fall 2014,"[{'value': 'AAS', 'descr': 'Asian American Stu...",AAS,Asian American Studies,Asian American Studies,[{'titleShort': 'Intro To Asian American Hist'...,Asians in the Americas,Asians in the Americas: A Comparative Perspective,The common perception of ethnicity is that it ...
2,2014,FA14,Fall 2014,"[{'value': 'AAS', 'descr': 'Asian American Stu...",AAS,Asian American Studies,Asian American Studies,[{'titleShort': 'Intro To Asian American Hist'...,Asian American Women's Hist,Asian American Women's History,This course examines the experiences and repre...
3,2014,FA14,Fall 2014,"[{'value': 'AAS', 'descr': 'Asian American Stu...",AAS,Asian American Studies,Asian American Studies,[{'titleShort': 'Intro To Asian American Hist'...,Independent Study,Independent Study,
4,2014,FA14,Fall 2014,"[{'value': 'AAS', 'descr': 'Asian American Stu...",AEM,Applied Economics & Management,Applied Economics & Management,[{'titleShort': 'FWS:Fd Systems in Devlpng Wrl...,FWS:Fd Systems in Devlpng Wrld,FWS:Food Systems In The Developing World: Heal...,What is it like to be a subsistence farmer in ...
5,2014,FA14,Fall 2014,"[{'value': 'AAS', 'descr': 'Asian American Stu...",AEM,Applied Economics & Management,Applied Economics & Management,[{'titleShort': 'FWS:Fd Systems in Devlpng Wrl...,Foundations of Entrep & Bus,Foundations of Entrepreneurship and Business,Introductory course providing a sound base to ...
6,2014,FA14,Fall 2014,"[{'value': 'AAS', 'descr': 'Asian American Stu...",AEM,Applied Economics & Management,Applied Economics & Management,[{'titleShort': 'FWS:Fd Systems in Devlpng Wrl...,Econ of Env & Nat Resources,An Introduction to the Economics of Environmen...,This course provides an introduction to the ec...
7,2014,FA14,Fall 2014,"[{'value': 'AAS', 'descr': 'Asian American Stu...",AEM,Applied Economics & Management,Applied Economics & Management,[{'titleShort': 'FWS:Fd Systems in Devlpng Wrl...,Cont Controversies in Global,Contemporary Controversies in the Global Economy,This course aims to stimulate critical thinkin...
8,2014,FA14,Fall 2014,"[{'value': 'AAS', 'descr': 'Asian American Stu...",AEM,Applied Economics & Management,Applied Economics & Management,[{'titleShort': 'FWS:Fd Systems in Devlpng Wrl...,Spreadsheet Modeling,Spreadsheet Modeling for Management and Economics,This course will be developed as a data-driven...
9,2014,FA14,Fall 2014,"[{'value': 'AAS', 'descr': 'Asian American Stu...",AEM,Applied Economics & Management,Applied Economics & Management,[{'titleShort': 'FWS:Fd Systems in Devlpng Wrl...,Introductory Statistics,Introductory Statistics,Introduces statistical methods. Topics include...
